In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import gymnasium as gym
from stable_baselines3.common.env_util import make_vec_env

import env as environment

pygame 2.5.2 (SDL 2.28.2, Python 3.8.17)
Hello from the pygame community. https://www.pygame.org/contribute.html


/home/bryan/miniconda3/envs/dreamer/lib/python3.8/site-packages/torch/utils/tensorboard/__init__.py:4: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if not hasattr(tensorboard, "__version__") or LooseVersion(
/home/bryan/miniconda3/envs/dreamer/lib/python3.8/site-packages/torch/utils/tensorboard/__init__.py:6: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  ) < LooseVersion("1.15"):
2023-11-26 09:53:49.123370: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-26 09:53:49.610839: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/bryan/miniconda3/envs/dreamer/lib/python3.8/site-packages/pandas/compat/nump

In [3]:
env = gym.make("SlidingEnv-v0")

In [6]:
a = [1, 2, 3]

/home/bryan/miniconda3/envs/dreamer/lib/python3.8/site-packages/gymnasium/envs/registration.py:694: UserWarning: WARN: Overriding environment SlidingEnv-v0 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")


In [8]:
a.remove(2)

In [9]:
a

[1, 3]

In [5]:
env.reset()

0.9888888888888889


/home/bryan/miniconda3/envs/dreamer/lib/python3.8/site-packages/gymnasium/envs/registration.py:694: UserWarning: WARN: Overriding environment SlidingEnv-v0 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")


(array([[ 1,  0,  2,  3],
        [ 4,  5,  6,  7],
        [ 8,  9, 10, 11],
        [12, 13, 14, 15]]),
 {})

In [ ]:
env = make_vec_env(
    env_id=config["env_name"],
    seed=config["seed"],
    wrapper_class=AtariWrapper,
    env_kwargs={"obs_type": "rgb"},
    n_envs=config["n_envs"],
)